# Lecture 23:  Cahn-Hilliard and Cahn-Allen Kinetics

### Sections

* [Introduction](#Introduction)
* [Learning Goals](#Learning-Goals)
* [On Your Own](#On-Your-Own)
    *  
* [In Class](#In-Class)
    *  
* [Homework](#Homework)
* [Summary](#Summary)
* [Looking Ahead](#Looking-Ahead)
* [Reading Assignments and Practice](#Reading-Assignments-and-Practice)

### Introduction
----

Cahn and Hilliard’s paper from 1958 appears in Journal of Chemical Physics v. 28, n. 2, p. 258-267.  They generalize the free energy of a system with composition gradients. They do so using a Taylor expansion. They develop a free energy functional and solve the resulting differential equation. Then a paper in 1961 titled "On Spinodal Decomposition" outlines the differential equation for the time rate of change of the composition in a spinodally decomposing system.

A number of items that I have discussed with you in this class are relevant to these papers:

* They develop a Taylor series approximation of the free energy.
* They make crystal symmetry arguments and consider rotations.
* They develop a functional representation of the free energy.
* They invoke the Beltrami identity to perform the first integral of the functional.
* They analytically solve the resulting differential equation to compute the surface energy of a non-homogeneous system.
* The fourth order differential equation is developed from a conservation law (such as Fick’s 2nd Law).

There are some difficulties with solving the resultant differential equation. Our goal is to simulate the process of spinodal decomposition using the Cahn-Hilliard equation - I will address the difficulties in passing and not provide any in-depth discussion for how we elminate the difficulties.

[Top of Page](#Sections)

### Learning Goals
----

* 
* 
* 
* 

[Top of Page](#Sections)

### On Your Own
----

[Top of Page](#Sections)

### In Class
----

#### An argument for the functional form

The Cahn-Hilliard equation is developed using a functional (i.e. calculus of variations) formalism.  We assume that the system of interest is a heterogenous system.  A heterogeneous system could be a system with spatial variations in composition or order.  It could also be a system where there are two phases or two different regions where the chemistry or chemical ordering is different.  We assume this system is isothermal and there is no volume change with changes in order or due to chemical gradients.

It is developed in Cahn's work to use the Helmholtz potential.  (This raises the question of why not use Gibbs potential?)  Different thermodynamic potentials are developed to mirror the experimental conditions we are trying to describe.  If we were examining a system at some fixed volume and temperature - the the Helmholtz potential is appropriate.  If you fix temperature and pressure, then Gibbs' potential is the most appropriate.  These potentials measure the maximum work you can extract subject to the constraints outlined.

We define the Helmholtz free energy for our system as:

$$
F = \int_V \left( f(\phi) + \frac{1}{2} \epsilon^2 (\nabla \phi)^2 \right) dV
$$

In Cahn's papers (and in other textbooks) the development of the free energy is more (and less) clearly spelled out.  The only requirement for us now is that the bulk free energy have a shape where the second derivative of the thermodynamic potential is negative.  One such function is:

$$
f(\phi) = W \phi^2 (1-\phi)^2
$$

where phi is some quantity that is either conserved (like concentration) or not conserved (like chemical ordering).

To develop the idea of the functional form further, it is necessary to think about the energetic contribution due to the hetereogeneous nature of the system.  In this formalism we penalize the system for gradients in the order parameter.  The penalty is given by the second term of the functional above.  This term arises on the basis of a Taylor expansion of the free energy in the order parameter and structural arguments.  For a comprehensive discussion on this development the student is directed to Cahn's paper.

It can be shown that the excess energy due to the gradient (and gradients due to the presence of interfaces) is mapped to the parameters $W$ and $\epsilon$ in the free energy functional.  These parameters, in turn can be formulated in terms of an interface excess energy and interfacial width.  I will not discuss this further here - but there is plenty of reading that can be consulted.

In [ ]:
%matplotlib notebook
import sympy as sp
from ipywidgets import interact
sp.init_session(quiet=True)

In [ ]:
phi, W, epsilon = symbols('phi W epsilon', real=true)

def doubleWell(ordParameter, wellHeight=1):
    """
    A small function that returns the double well shape.
    """
    return wellHeight*ordParameter**2*(1-ordParameter)**2

doubleWell(phi)

In [ ]:
sp.plot(doubleWell(phi, 1.0), (phi,-0.1,1.1), ylim=(0,0.1));

[Top of Page](#Sections)

#### The Equations of Motion

With the bulk free energy and the gradient energy contributions conceptually justified it is now necessary to identify the equations of motion.  In the non conserved case:

$$
\frac{\partial \phi}{\partial t} = -M \frac{\delta F}{\delta \phi}
$$

and for a conserved order parameter the equations of motion are derived from:

$$
\frac{\partial \phi}{\partial t} = \nabla \cdot D \nabla \frac{\delta F}{\delta \phi}
$$

These are best guesses for how the kinetics should go.  There are other choices, but these are the simplest choices that guarantee a free energy decrease with time.

[Top of Page](#Sections)

#### The Functional Derivative

We will invoke the Calculus of Variations and `SymPy` to develop the functional derivative.  We start by writing down the functional form:

In [ ]:
functionalForm = W*phi(x)**2*(1-phi(x))**2 + sp.Rational(1,2)*epsilon*(phi(x).diff(x))**2
functionalForm

We use `SymPy`'s built in function for taking a variational derivative:

In [ ]:
ele = sp.euler_equations(functionalForm, phi(x), x)
ele

What we get back can be factored a bit more:

In [ ]:
delFdelPhi = (ele[0].lhs).simplify()
delFdelPhi

The print command helps a bit when you only want to grab a few terms (there are probably more programmatic ways to do this):

In [ ]:
print(delFdelPhi)

Now we factor those terms:

In [ ]:
firstTermsFactored = sp.factor(4*W*phi**3-6*W*phi**2+2*W*phi)
firstTermsFactored

The simplified functional derivative is therefore:

$$
\frac{\delta F}{\delta \phi} = 2 W \phi \left(\phi - 1\right) \left(2 \phi - 1\right) - \epsilon \frac{d^{2}}{d x^{2}}  \phi{\left (x \right )}
$$

[Top of Page](#Sections)

#### A Slightly Simpler Conservation Form

If the equation of motion is for a non conserved variable - then we're almost done.  All that remains is to specify the mobility $M$ or diffusion coefficient $D$ and write the equations of motion.

When writing the equations of motion - things can get messy.  It is better therefore to write the leading term on the LHS as $A(\phi)$.  This gives:

$$
\frac{\delta F}{\delta \phi} = A(\phi) - \epsilon \frac{d^{2}}{d x^{2}}  \phi{\left (x \right )}
$$


Making:

$$
\frac{\partial \phi}{\partial t} = \nabla \cdot D \nabla \frac{\delta F}{\delta \phi}
$$

with

$$
\nabla \cdot D \nabla \frac{\delta F}{\delta \phi} = \nabla \cdot D \left( \frac{\partial A}{\partial \phi} \nabla \phi(x) - \epsilon \frac{d^{3}}{d x^{3}} \phi(x) \right)
$$

By distributing the divergence and diffusion coefficient, we arrive at:

$$
\frac{\partial \phi}{\partial t} = \nabla \cdot D \frac{\partial A}{\partial \phi} \nabla \phi(x) - D \epsilon \nabla^4 \phi(x)
$$

The first term on the LHS contains a $\phi$ dependent diffusion coefficient and the second term is a "fourth order" diffusion coefficient.  In FiPy we can represent terms such as:

$$\nabla \cdot \left\{ D_1 \nabla \left[ \nabla\cdot\left( D_2 \nabla \phi\right) \right] \right\}$$

using the standard `DiffusionTerm` with a tuple as the diffusion coefficient.  In our case each of the diffusion coefficients are constant with respect to $\phi$.

[Top of Page](#Sections)

#### Solving the Cahn-Hilliard Equation

In [ ]:
%matplotlib
from fipy import *
nx = ny = 50

mesh = Grid2D(nx=nx, ny=ny, dx=0.50, dy=0.50)
phi = CellVariable(name=r"$\phi$", mesh=mesh)

noise = GaussianNoiseVariable(mesh=mesh,mean=0.5,variance=0.01).value
phi[:] = noise

viewer = Viewer(vars=phi, datamin=-0.1, datamax=1.1)

D = a = epsilon = 1.

PHI = phi.arithmeticFaceValue

eq = (TransientTerm() == \
      + DiffusionTerm(coeff=D * a**2 * (1 - 6 * PHI * (1 - PHI))) \
      - DiffusionTerm(coeff=(D, epsilon**2))) 

dexp = -5
elapsed = 0.
duration = 10000

while elapsed < duration:
    dt = min(1, numerix.exp(dexp))
    elapsed += dt
    dexp += 0.05
    eq.solve(dt=dt, var=phi)
    viewer.plot()

#### Solving the Cahn-Allen Equation

In [ ]:
%matplotlib
from fipy import *
nx = ny = 50

mesh = Grid2D(nx=nx, ny=ny, dx=0.50, dy=0.50)
phi = CellVariable(name=r"$\phi$", mesh=mesh)

noise = GaussianNoiseVariable(mesh=mesh,mean=0.5,variance=0.01).value
phi[:] = noise

viewer = Viewer(vars=phi, datamin=-0.1, datamax=1.1)

D = a = epsilon = 1.

PHI = phi.arithmeticFaceValue

S0 = 4*a*phi**3 - 6*a*phi**2 + 2*a*phi
S1 = 4*a*phi**2 - 6*a*phi**1 + 2*a

eq = (TransientTerm(var=phi) ==  - S0 + DiffusionTerm(var=phi, coeff=epsilon)) 

dexp = -5
elapsed = 0.
duration = 10000

while elapsed < duration:
    dt = min(1, numerix.exp(dexp))
    elapsed += dt
    dexp += 0.05
    eq.solve(dt=dt, var=phi)
    viewer.plot()
    
    
    

[Top of Page](#Sections)

### Homework
----

[Top of Page](#Sections)

### Looking Ahead
----

[Top of Page](#Sections)

### Reading Assignments and Practice
----

[Top of Page](#Sections)